# \_\_slots\_\_

- 파이썬은 인스턴스의 모든 속성을 dict에 저장
- \_\_slots\_\_ 명령어를 통해 속성을 직접 정의
- 속성 정의/호출 속도 향상 및 메모리 사용량 감소
- 대신, 정의 이후 속성 추가 또는 제거 불가

참고: https://blog.usejournal.com/a-quick-dive-into-pythons-slots-72cdc2d334e

In [1]:
class MyClass(object):
    __slots__ = ['name', 'identifier'] # __slots__ 추가
    def __init__(self, name, identifier):
        self.name = name
        self.identifier = identifier
        self.set_up()

In [2]:
import ipython_memory_usage.ipython_memory_usage as imu

In [3]:
num = 1024*256

In [4]:
imu.start_watching_memory()

In [4] used 0.0273 MiB RAM in 0.12s, peaked 0.00 MiB above current, total RAM usage 60.41 MiB


In [5]:
class MyClass1(object):
        __slots__ = ['name', 'identifier']
        def __init__(self, name, identifier):
                self.name = name
                self.identifier = identifier

In [5] used 0.1016 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 60.51 MiB


In [6]:
%timeit [MyClass1(1,1) for i in range(num)]

76.7 ms ± 892 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
In [6] used 1.1797 MiB RAM in 6.36s, peaked 14.73 MiB above current, total RAM usage 61.69 MiB


In [7]:
x = [MyClass1(1,1) for i in range(num)]

In [7] used 14.8633 MiB RAM in 0.26s, peaked 0.00 MiB above current, total RAM usage 76.55 MiB


In [8]:
class MyClass2(object):
        def __init__(self, name, identifier):
                self.name = name
                self.identifier = identifier

In [8] used 0.0820 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 76.63 MiB


In [9]:
%timeit [MyClass2(1,1) for i in range(num)]

105 ms ± 2.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
In [9] used 0.3438 MiB RAM in 8.69s, peaked 45.12 MiB above current, total RAM usage 76.98 MiB


In [10]:
y = [MyClass2(1,1) for i in range(num)]

In [10] used 45.1836 MiB RAM in 0.27s, peaked 0.00 MiB above current, total RAM usage 122.16 MiB
